In [2]:
from opensearchpy import OpenSearch
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import random
import datetime
import boto3
import os


In [3]:
######################### 학습데이터 날짜범위
now = datetime.datetime.now()
today = now.strftime("%Y-%m-%d")
before_2month = now - datetime.timedelta(days=2*30)
before_2month = before_2month.strftime("%Y-%m-%d")

In [6]:
######################### S3 채널 리스트 불러오기
AWS_ACCESS_KEY_ID= os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY= os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION_NAME = os.getenv("AWS_REGION_NAME")
S3_BUCKET= os.getenv("S3_BUCKET")
CHANNEL_LIST_PATH= os.getenv("CHANNEL_LIST_PATH")

s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID, \
                        aws_secret_access_key=AWS_SECRET_ACCESS_KEY, \
                        region_name=AWS_REGION_NAME)
# S3에서 파일 가져오기
response = s3.get_object(Bucket=S3_BUCKET,   \
                        Key=CHANNEL_LIST_PATH)

file_content = response['Body'].read().decode('utf-8')
use_channels = file_content[1:].split("\n")

['UC2untV6RHYn-FMJe1JZyi-g',
 'UCwNS7NBBxDIYtTi3rnPONvg',
 'UCftXk2n3io6hMik2uP-0_1Q',
 'UCDX5PcGdh73bnrVMgdQ_DoA',
 'UCqsoQi0NXL7la-8cBQExy_Q',
 'UCdp4_yTBhQmB8E339Lafzow',
 'UCyuKUmRH3MiHmbN4rh5alXw',
 'UC-huoa_cK8gHFHFjku-jyoQ',
 'UCSNQapL0dCkVX2IiQzPASIw',
 'UCqWBDPCLCgMMDo47dAHyihw',
 'UCI7OwvW0AJSXW92AcJoBhPw',
 'UCPOugQkPqc4NSgm6eEn-EOg',
 'UCjJlJ5sUAOGXIp7TJbj_oKQ',
 'UCDJwGnwK8bBhsfkmVnwcfLw',
 'UCI7OFjGUtM8yePASCBdnUUg',
 'UCQUTs-VRcWF9lVJUsu3vfcA',
 'UCbD8EppRX3ZwJSou-TVo90A',
 'UC1t-vzAacQ4OiMLt5h7Flug',
 'UCrXcXpE72U2IHAxuoiQwLIQ',
 'UCM1NcK-PXAmxHuI_1cpKbGQ',
 'UCYeBKS__kDnCFnv9SXK2-9g',
 'UC9oojNQTAvb2SEsX4jYjBvg',
 'UCjfFwPbCtUdWNu1chY4ebEw',
 'UCVO5SKRlFzSM9lGpAI2SGsw',
 'UCzcnCnQ7IxmCAP77v1J6M9Q',
 'UCwlixQLZpKejReLsE7pr0Og',
 'UC-NbaQnwEJWZHWQWKLVgZ-A',
 'UCwNVDlUgdBlmVWRPy4Nk0Dw',
 'UC7LN0FW9hvkov8Sf746yW9A',
 'UCURffR0e_FPKnDuzQ0XyWhw',
 'UCo567TEELBhGxRsM4hYm0YA',
 'UCKrwt-cJtfuvhKxf-seiWvQ',
 'UCDULrK2OJsiDhFroa2Aj_LQ',
 'UCNnTPPw5iGu0A0-2HF2GpcQ',
 'UC5drpy89Sev

In [7]:
# OpenSearch 클라이언트 객체 생성

OPENSEARCH_HOST= os.getenv("OPENSEARCH_HOST")
OPENSEARCH_ID= os.getenv("OPENSEARCH_ID")
OPENSEARCH_PW= os.getenv("OPENSEARCH_PW")


client = OpenSearch(
    hosts=[OPENSEARCH_HOST],
    http_auth="%s:%s"%(OPENSEARCH_ID, OPENSEARCH_PW),
    scheme="https",
    verify_certs=False,
    timeout=30
)


/usr/local/lib/python3.8/dist-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://search-dothis-js7jgo2actyuzihx7zz335k2nq.ap-northeast-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [8]:
# 검색할 쿼리 작성
# query = {
#     "query": {
#         "range": {
#             "video_published": {
#                 "gte": "2019-04-30",
#                 "lte": "2023-04-30"}
#         }
#     },
#     "size":10000
# }


# 사용컬럼
# use_col = ["video_id", "channel_id", "video_title", "video_description", "video_tag", "video_published"]

# # 전체 가져오기
# query = {
#     "query": {
#         "match_all": {
#         }
#     },
#     "_source": use_col,
#     "size":1000
# }



# 사용컬럼
use_col = ["video_id", "channel_id", "video_title", "video_description", "video_tag", "video_published"]

# 검색할 쿼리 작성
query = {
    "query": {
        "bool": {
            "filter": [
                {"terms": {"channel_id.keyword": use_channels}},
                {"range": {"video_published": {"gte": before_2month, "lte": today}}}
            ]
        }
    },
    "_source": use_col,
    "size": 10000
}



In [9]:
# 검색 실행 및 결과 가져오기
response = client.search(index="video", body=query, scroll='1m')

# 검색 결과를 처리합니다.
scroll_id = response['_scroll_id']
# total_hits = response['hits']['total']['value']
scroll_size = len(response['hits']['hits'])

df = pd.DataFrame(columns=use_col)

while scroll_size > 0:
    # 현재 페이지의 검색 결과를 처리합니다.
    # for hit in response['hits']['hits']:
    #     df.loc[len(df)] = hit['_source']

    hits = response['hits']['hits']
    df = pd.concat([df, pd.DataFrame([hit['_source'] for hit in hits])], ignore_index=True)
    
    # 다음 페이지의 검색 결과를 가져옵니다.
    response = client.scroll(scroll_id=scroll_id, scroll='1m')
    scroll_id = response['_scroll_id']
    scroll_size = len(response['hits']['hits'])
    
df = df.sort_values('video_published').reset_index(drop=True)
df

/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-dothis-js7jgo2actyuzihx7zz335k2nq.ap-northeast-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-dothis-js7jgo2actyuzihx7zz335k2nq.ap-northeast-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,video_id,channel_id,video_title,video_description,video_tag,video_published
0,_0wFLukEhXk,UC2_Thm67UK6azzECZ-_bIxw,(문풀) 쎈 고등 수학 (1) - B단계: 207번 (32p),쎈 고등 수학 (1) \n32 페이지\nB단계\n유형12. 이차방정식과 로그\n난이...,[],2023-03-28
1,_2pn7CUABn8,UC-0C8yVGJy-cS4FGlYKelWw,특파원보고 세계는 지금 예고 (KBS_304회_2023.04.08.방송 예고) 토요...,"■ 美 첫 형사 기소 대통령 트럼프\n지난 4월 4일(현지시각), 전·현직 포함 미...","['특파원보고 세계는 지금', '세계는 지금', '국제시사']",2023-04-06


In [57]:
from nlp import *
use_texts = df.progress_apply(lambda x: preprocessing_text(x["video_title"], \
                                                            use_morphs=False, \
                                                            use_norns=True, \
                                                            delete_len_1=True) \
                                + " " + preprocessing_text(x["video_tag"], \
                                                            use_morphs=False, \
                                                            use_norns=False, \
                                                            delete_len_1=True) \
                                + " " + hashtag_extraction(x["video_description"]) \
                                , axis=1)


use_texts.tolist()

100%|██████████| 5/5 [00:00<00:00, 2498.39it/s]


['VLOG 현명 주식 투자 경제 자유 중요  경제적자유 경제 재테크',
 '투자 SCHD 가미 배당 장형 변형 웨더 포트폴리오  재테크 인플레이션 미국주식',
 '행복 음악 음악 스트레스 해소 음악 음악 Good Night 수면음악 수면유도음악 잔잔한음악 잠오는음악 10시간 수면음악 10시간 스트레스해소음악 편안한음악 희망적인 음악 불면증음악 불면증해소음악 잠잘오는음악 잠잘때듣는음악 ',
 '고등 수학 단계  ',
 '특파원 세계 예고 KBS 방송 예고 토요일 KBS TV 방송 특파원보고 세계는 지금 세계는 지금 국제시사 ']

In [8]:


use_channel_id = random.sample(list(df["channel_id"].unique()), 500)
print(len(use_channel_id), use_channel_id)

path = "./data/train_channel_list_test.txt"
f = open(path, "w", encoding='utf-8-sig')
for i in use_channel_id:
    f.write(i+'\n')
f.close()

500 ['UC2untV6RHYn-FMJe1JZyi-g', 'UCwNS7NBBxDIYtTi3rnPONvg', 'UCftXk2n3io6hMik2uP-0_1Q', 'UCDX5PcGdh73bnrVMgdQ_DoA', 'UCqsoQi0NXL7la-8cBQExy_Q', 'UCdp4_yTBhQmB8E339Lafzow', 'UCyuKUmRH3MiHmbN4rh5alXw', 'UC-huoa_cK8gHFHFjku-jyoQ', 'UCSNQapL0dCkVX2IiQzPASIw', 'UCqWBDPCLCgMMDo47dAHyihw', 'UCI7OwvW0AJSXW92AcJoBhPw', 'UCPOugQkPqc4NSgm6eEn-EOg', 'UCjJlJ5sUAOGXIp7TJbj_oKQ', 'UCDJwGnwK8bBhsfkmVnwcfLw', 'UCI7OFjGUtM8yePASCBdnUUg', 'UCQUTs-VRcWF9lVJUsu3vfcA', 'UCbD8EppRX3ZwJSou-TVo90A', 'UC1t-vzAacQ4OiMLt5h7Flug', 'UCrXcXpE72U2IHAxuoiQwLIQ', 'UCM1NcK-PXAmxHuI_1cpKbGQ', 'UCYeBKS__kDnCFnv9SXK2-9g', 'UC9oojNQTAvb2SEsX4jYjBvg', 'UCjfFwPbCtUdWNu1chY4ebEw', 'UCVO5SKRlFzSM9lGpAI2SGsw', 'UCzcnCnQ7IxmCAP77v1J6M9Q', 'UCwlixQLZpKejReLsE7pr0Og', 'UC-NbaQnwEJWZHWQWKLVgZ-A', 'UCwNVDlUgdBlmVWRPy4Nk0Dw', 'UC7LN0FW9hvkov8Sf746yW9A', 'UCURffR0e_FPKnDuzQ0XyWhw', 'UCo567TEELBhGxRsM4hYm0YA', 'UCKrwt-cJtfuvhKxf-seiWvQ', 'UCDULrK2OJsiDhFroa2Aj_LQ', 'UCNnTPPw5iGu0A0-2HF2GpcQ', 'UC5drpy89Sev-YLulkOfT8Wg', 'UCeF2KcMHIuYqQ